In [1]:
!python -V

Python 3.10.16


In [2]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [3]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("my-first-experiment")

<Experiment: artifact_location='/home/shiguita/Desktop/projects/01-Mlops-zoomcamp/notebooks/week-2/mlruns/2', creation_time=1748406567050, experiment_id='2', last_update_time=1748406567050, lifecycle_stage='active', name='my-first-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_parquet(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
jan_2025 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-01.parquet'
feb_2025 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-02.parquet'

df_train = read_dataframe(jan_2025)
print(f'Train Dataset dimensions -> {df_train.shape=}')

df_val = read_dataframe(feb_2025)
print(f'Validation Dataset dimensions -> {df_val.shape=}')

Train Dataset dimensions -> df_train.shape=(46307, 22)
Validation Dataset dimensions -> df_val.shape=(44218, 22)


In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

6.098706133427926

In [10]:
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))

with open(f'{parent_dir}/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [11]:
with mlflow.start_run():

    mlflow.set_tag("developer", "sh")

    mlflow.log_param("train-data-path", jan_2025)
    mlflow.log_param("val-data-path", feb_2025)

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path=f'{parent_dir}/models/lin_reg.bin', artifact_path="models_pickle")

In [12]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [18]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/shiguita/anaconda3/envs/experiment-tracking-env/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [00:04:15] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.79399                           
[1]	validation-rmse:8.50469                           
[2]	validation-rmse:8.23745                           
[3]	validation-rmse:7.99015                           
[4]	validation-rmse:7.76190                           
[5]	validation-rmse:7.55214                           
[6]	validation-rmse:7.35883                           
[7]	validation-rmse:7.18126                           
[8]	validation-rmse:7.01836                           
[9]	validation-rmse:6.86912                           
[10]	validation-rmse:6.73316                          
[11]	validation-rmse:6.60830                          
[12]	validation-rmse:6.49432                          
[13]	validation-rmse:6.39112                          
[14]	validation-rmse:6.29673                          
[15]	validation-rmse:6.21037                          
[16]	validation-rmse:6.13283                          
[17]	validation-rmse:6.06127                          
[18]	valid

/home/shiguita/anaconda3/envs/experiment-tracking-env/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [00:06:00] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.81378                                                     
[1]	validation-rmse:5.43743                                                     
[2]	validation-rmse:5.38359                                                     
[3]	validation-rmse:5.38038                                                     
[4]	validation-rmse:5.38634                                                     
[5]	validation-rmse:5.38453                                                     
[6]	validation-rmse:5.38173                                                     
[7]	validation-rmse:5.37684                                                     
[8]	validation-rmse:5.37676                                                     
[9]	validation-rmse:5.37942                                                     
[10]	validation-rmse:5.38126                                                    
[11]	validation-rmse:5.38289                                                    
[12]	validation-rmse:5.38341

/home/shiguita/anaconda3/envs/experiment-tracking-env/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [00:06:09] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.78763                                                    
[1]	validation-rmse:8.49481                                                    
[2]	validation-rmse:8.22257                                                    
[3]	validation-rmse:7.97585                                                    
[4]	validation-rmse:7.74579                                                    
[5]	validation-rmse:7.53898                                                    
[6]	validation-rmse:7.34582                                                    
[7]	validation-rmse:7.17432                                                    
[8]	validation-rmse:7.01380                                                    
[9]	validation-rmse:6.86847                                                    
[10]	validation-rmse:6.73547                                                   
[11]	validation-rmse:6.61966                                                   
[12]	validation-rmse:6.50473            

/home/shiguita/anaconda3/envs/experiment-tracking-env/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [00:08:09] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.75756                                                    
[1]	validation-rmse:8.43801                                                    
[2]	validation-rmse:8.14570                                                    
[3]	validation-rmse:7.87798                                                    
[4]	validation-rmse:7.63375                                                    
[5]	validation-rmse:7.41122                                                    
[6]	validation-rmse:7.20857                                                    
[7]	validation-rmse:7.02504                                                    
[8]	validation-rmse:6.85896                                                    
[9]	validation-rmse:6.70855                                                    
[10]	validation-rmse:6.57279                                                   
[11]	validation-rmse:6.45061                                                   
[12]	validation-rmse:6.34059            

KeyboardInterrupt: 

In [20]:
import mlflow.xgboost


best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

mlflow.xgboost.autolog(disable=False)
booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=10,
    evals=[(valid, 'validation')],
    early_stopping_rounds=10
)

2025/05/28 00:12:08 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 3.0.0, but the installed version is 3.0.2. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2025/05/28 00:12:08 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ca66791a53764192bdf15ba39734b7d1', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/shiguita/anaconda3/envs/experiment-tracking-env/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [00:12:11] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:8.56745
[1]	validation-rmse:8.10033
[2]	validation-rmse:7.69739
[3]	validation-rmse:7.35105
[4]	validation-rmse:7.05239
[5]	validation-rmse:6.80020
[6]	validation-rmse:6.58162
[7]	validation-rmse:6.39726
[8]	validation-rmse:6.24197
[9]	validation-rmse:6.11059


2025/05/28 00:12:37 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/28 00:12:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/shiguita/anaconda3/envs/experiment-tracking-env/lib/python3.10/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [00:12:37] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/28 00:12:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [21]:
mlflow.xgboost.autolog(disable=True)

In [22]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=10,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/shiguita/anaconda3/envs/experiment-tracking-env/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [00:15:22] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:8.56745
[1]	validation-rmse:8.10033
[2]	validation-rmse:7.69739
[3]	validation-rmse:7.35105
[4]	validation-rmse:7.05239
[5]	validation-rmse:6.80020
[6]	validation-rmse:6.58162
[7]	validation-rmse:6.39726
[8]	validation-rmse:6.24197
[9]	validation-rmse:6.11059


FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
